In [81]:
#############################
##### the hanabi game #######
#############################

### class: player, discard, played, leftover
### action/decision: (1) give info (2) play card (3) discard

from scipy import *
cards_ordered = array([[num, color] for num in [1,1,1,2,2,3,3,4,4,5] for color in range(1,6)])

In [127]:
#############################
#### the deck class #########
#############################

class deck:
    def __init__(self, cards=zeros(1)):
        if cards.any():
            self._data = cards
            self._knowledge = zeros(shape=cards.shape).astype(int)
        else:
            self._data = array([]).reshape(-1,2).astype(int)
            self._knowledge = array([]).reshape(-1,2)
    def add (self, icard):
        self._data = append(self._data, icard.astype(int)).reshape(-1,2)
    def rm (self, idx):
        self._data = delete(self._data, idx, axis=0)
    def showcard (self):
        return self._data
    def updateknowledge (self, idx, pos=0): ## pos 0 for num, pos 1 for color
        self._knowledge[idx, pos] = 1
    def showknowledge (self):
        return self._knowledge
    def match (self, icard):
        return where((self._data.T[0] == icard[0]) & (self._data.T[1] == icard[1]))[0]
    def ncard (self):
        return len(self._data)
    
#jia = player() 
#jia = player(cards_init[:4])
#print jia.showcard()
#print jia.showknowledge()

In [146]:
###################################
####### initialize the game #######
###################################

class hanabi:
    def __init__(self, N, iseed=10027):
        '''Initialize the game, with N players, and seed to shuffle the card'''
        seed(iseed)
        order = range(50)
        shuffle(order)
        self._cards_init = cards_ordered [order]
        self._fuse = 4
        self._info = 8
        ###### set up the initial N player decks
        ###### with leftover (no knowledge), played (full), discard pile (full)
        self._leftover = deck(self._cards_init)
        self._played = deck(array([[num, color] for num in range(1,6) for color in range(1,6)]))
        self._discarded = deck()
        # print self.leftover.showcard()
        self._players = []
        for iN in range(N): ## initiate N deck objects
            _newcard = self._leftover.showcard()[:N]
            self._players.append (deck(_newcard))
            self._leftover.rm(range(N))
            # print self.leftover.ncard(), iN, players[iN].showcard()
    def N (self):
        return N
    def fuse(self,burn=0):
        if burn==1:
            self._fuse -= 1
        else:
            return self._fuse
    def info(self, update=0):
        if update:
            self._info += update
        else:
            return self._info
    def leftover(self):
        return self_leftover
    def draw(self, iN): #remove 1 from leftover, iN player get a new card
        _newcard = self._leftover.showcard()[0]
        self._players[iN].add(_newcard)
        self._leftover.rm(0)
    def discard(self, iN, n, draw=1): # discard the nth card in iN player deck and draw a new card
        _icard=self._players[iN].showcard()[n]
        self._players[iN].rm(n) ## remove the card from player hand
        self._discarded.add(_icard) ## put this card to discard pile    
        self.info(1)
        if draw:
            self.draw(iN)
    def discarded(self):
        return self._discarded    
    def reaction_to_play (self, _icard):
        '''for a new played card, decide its destination play or discard'''
        idx_match = self._played.match(_icard) ## find idx of the newcard in played pile
        if sum(self._played.showknowledge()[idx_match]): 
            ### discard if newcard already in played
            self._discarded.add(_icard)
            self._fuse -= 1 ## penalty due to wrong card
        else:
            if _icard[0]==1: 
                ## play, if this is a #1 card
                print 'boo'
                self._played.updateknowledge(idx_match,pos=0)
            else: 
                ## check if the card before is played
                idx_before = self._played.match(_icard-array([1,0]))
                if self._played.showknowledge()[idx_before].any():# play, the card before is played
                    self._played.updateknowledge(idx_match,pos=0)
                else: ## discard if the card before is not played
                    self._discarded.add(_icard)
                    self._fuse -= 1
    def play(self, iN, n,draw=1): # play one card, update played, draw a new card
        _icard=self._players[iN].showcard()[n]
        self._players[iN].rm(n)        
        self.reaction_to_play (_icard) # do fuse, played/discarded
        if draw:
            self.draw(iN)
    def played(self):
        return self._played
    def players(self):
        return self._players


In [149]:
test_hanabi_class = 0
if test_hanabi_class: ####### pass
    game1 = hanabi(4)
    print 'num of players:',game1.N()
    print 'player1 card:\n',game1.players()[1].showcard()
    #print 'player1 knowledge:\n',game1.players()[1].showknowledge()
    print 'discarded:',game1.discarded().showcard()
    print 'fuse, info:',game1.fuse(), game1.info()

    print '===== discard last card in player 1, check again =========='
    game1.discard(1,3)
    print 'player1 card:\n',game1.players()[1].showcard()
    #print 'player1 knowledge:\n',game1.players()[1].showknowledge()
    print 'discarded:',game1.discarded().showcard()
    print 'fuse, info:',game1.fuse(), game1.info()

    print '===== discard last card in player 1, 2nd time, check again =========='
    game1.discard(1,3)
    print 'player1 card:\n',game1.players()[1].showcard()
    #print 'player1 knowledge:\n',game1.players()[1].showknowledge()
    print 'discarded:\n',game1.discarded().showcard()
    print 'fuse, info:',game1.fuse(), game1.info()

    print '===== manually add a [1,1] card then play, check player, played, fuse, info'
    game1.discard(1,3,draw=0)
    print 'player1 card (discard the last card, no draw):\n',game1.players()[1].showcard()
    game1.players()[1].add(ones(2))
    print 'player1 card (draw a new card [1,1]):\n',game1.players()[1].showcard()
    game1.play(1,3)
    print 'player1 card (play [1,1]):\n',game1.players()[1].showcard()

    print 'discarded:\n',game1.discarded().showcard()
    print 'fuse, info:',game1.fuse(), game1.info()
    print 'played card:\n', game1.played().showcard() *game1.played().showknowledge()


In [ ]:
###########################
### game starts ###########
###########################

discard = deck()           
played = deck(array([[num, color] for num in range(1,6) for color in range(1,6)]))
fuse = 4
info = 8

#while leftover.ncard() > 0 and fuse >0:

In [84]:
###########################
### make decisions ########
###########################

def ratecard (played, discard, otherplayers, icard):
    '''With the knowledge of played, discard, other players card, 
    decide if icard (with information) is important.
    Input: played, discard, otherplayers, icard
    Return: Importance rating where 
        3 - very important, either a 5, or the only card left to complete the set
        2 - somewhat important, have 2 of these cards left
        1 - not important, is a 1 card and have 2 other cards
        0 - discard already played, or completed, or failed color'''

def decision (played, players, N_current, discard, info):
    #### first, when there is no info point, discard or play a card
    if info == 0:
        return 0
        ####### check the importance of the card
    #### if there are info, decide to play card or give info 